In [1]:
## Import dependencies

import numpy as np
from os import path
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim
import copy
import time


# Set default plot size
plt.rcParams["figure.figsize"] = (30,20)

# Define number of epochs used later in training
num_epochs = 500

# Identification part of the filenames
base_name = '50000spec_Combined1Distribution'base_dir = '/path/to/base/directory'   # Set base directory

## Train CNN on dataset of 44 metabolites

In [2]:
# Name variable used for saving model metrics, name should reflect model used, dataset used, and other information such as # of epochs
ModelName = f"CNN_44met_{base_name}Dist_TrainingAndValidation_ForManuscript_" + str(num_epochs) +"ep"

# Set the random seed
os.chdir(base_dir+'/DL-NMR-Optimization/ModelPerformanceMetrics/') 
seed = 1
torch.manual_seed(seed)
np.save(ModelName + "_Seed.npy", seed)

In [3]:
## Load training and testing datasets, validation datasets, and representative example spectra 

# Switch to directory containing datasets
os.chdir(base_dir+'/DL-NMR-Optimization/GeneratedDataAndVariables')

# Load training data and max value from testing and training datasets
spectra = np.load(f'Dataset44_{base_name}_ForManuscript_Spec.npy')
conc1 = np.load(f'Dataset44_{base_name}_ForManuscript_Conc.npy')

In [4]:
## Prepare to switch data from CPU to GPU

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    device = torch.device("cuda")          # A CUDA device object
    print("Using GPU for training.")
else:
    device = torch.device("cpu")           # A CPU object
    print("CUDA is not available. Using CPU for training.")

Using GPU for training.


In [5]:
## Set up data for testing and training

# Split into testing and training data
X_train, X_test, y_train, y_test = train_test_split(spectra, conc1, test_size = 0.2, random_state = 1)

# Tensorize and prepare datasets
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()


# Move the input data to the GPU device
X_train = X_train.to(device)
X_test = X_test.to(device)
#spectraVal = torch.tensor(spectraVal).float().to(device)   # Confusing names, these spectra are the 5000 spectra generated like the training dataset
#ValSpectra = torch.tensor(ValSpectra).float().to(device)   # Confusing names, these spectra are the 10 representative example spectra

# Move the target data to the GPU device
y_train = y_train.to(device)
y_test = y_test.to(device)
#concVal = torch.tensor(concVal).float().to(device)
#ValConc = torch.tensor(ValConc).float().to(device)

# More data prep?
datasets = torch.utils.data.TensorDataset(X_train, y_train)
Test_datasets = torch.utils.data.TensorDataset(X_test, y_test)
train_dataset_reshaped = [(data.unsqueeze(1), label) for data, label in datasets]
test_dataset_reshaped = [(data.unsqueeze(1), label) for data, label in Test_datasets]
train_iter = torch.utils.data.DataLoader(train_dataset_reshaped, batch_size = 64, shuffle=True)
test_iter = torch.utils.data.DataLoader(test_dataset_reshaped, batch_size = 64, shuffle=True)

In [6]:
del X_train
del X_test
del y_train
del y_test
del spectra
del conc1
del datasets
del Test_datasets

In [7]:
## Define NN model object, define some parameters, and instantiate model

# Define some model & training parameters
criterion = nn.MSELoss(reduction='sum')


# Define model
class NMR_Model_Aq(nn.Module):
    def __init__(self):
        super(NMR_Model_Aq, self).__init__()
        self.conv1 = nn.Conv1d(1, 42, kernel_size=6, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        self.conv2 = nn.Conv1d(42, 42, kernel_size=6, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        self.conv3 = nn.Conv1d(42, 42, kernel_size=6, padding=1)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        self.conv4 = nn.Conv1d(42, 42, kernel_size=6, padding=1)
        self.pool4 = nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(120708, 200)
        self.fc2 = nn.Linear(200, 44)

    def forward(self, x):
        x = x.permute(0, 2, 1)                  
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.pool3(x)
        x = self.conv4(x)
        x = nn.functional.relu(x)
        x = self.pool4(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x

    

In [8]:
class RelativeAbsoluteError(nn.Module):
    def __init__(self):
        super(RelativeAbsoluteError, self).__init__()

    def forward(self, y_pred, y_true):
        # Compute the mean of the true values
        y_mean = torch.mean(y_true)
        
        # Compute the absolute differences
        absolute_errors = torch.abs(y_true - y_pred)
        mean_absolute_errors = torch.abs(y_true - y_mean)
        
        # Compute RAE
        rae = torch.sum(absolute_errors) / torch.sum(mean_absolute_errors)
        return rae

In [9]:
def train_and_save_best_model(model, train_loader, test_loader, num_epochs, save_path):
    criterion = RelativeAbsoluteError()
    optimizer = optim.AdamW(model.parameters(), lr = 5.287243368897864e-05, weight_decay=0.01)
    
    
    train_losses = []
    test_losses = []
    best_test_loss = float('inf')
    epochs_no_improve = 0
    patience = 50  # Set how many epochs without improvement in validation loss constitutes early stopping

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        train_losses.append(train_loss)

        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for inputs, targets in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item() * inputs.size(0)
            test_losses.append(test_loss)
            
            
        if (epoch + 1) % 1 == 0:  # The last number here denotes how often to print loss metrics in terms of epochs
            print(f'Epoch [{epoch + 1}/{num_epochs}], '
                  f'Train Loss: {train_loss:.4f}, '
                  f'Test Loss: {test_loss:.4f}')
            
    
            
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            epochs_no_improve = 0
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, save_path)
        else:
            epochs_no_improve += 1
            
        if epochs_no_improve >= patience:
            print(f'Early stopping at epoch {epoch + 1}')
            break



    return train_losses, test_losses


def train_or_load_model(model, train_loader, test_loader, num_epochs, save_path):
    train_losses = []
    test_losses = []
    is_model_trained = False  # Initialize flag

    if os.path.isfile(save_path):
        print("Loading pretrained model from {}".format(save_path))
        checkpoint = torch.load(save_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer = optim.Adam(model.parameters())  
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        
    
    else:
        print("No pretrained model found. Training from scratch.")
        #optimizer = optim.Adam(model.parameters())  
        train_losses, test_losses = train_and_save_best_model(model, train_loader, test_loader, num_epochs, save_path)
        is_model_trained = True  # Set flag to True after training
        # Save losses per epoch
        np.save(ModelName + "_TrainLoss.npy", train_losses)
        np.save(ModelName + "_TestLoss.npy", test_losses)
    
    return train_losses, test_losses, is_model_trained  # Return the losses and flag

In [10]:
## Instantiate model and train

# For timing cell run time
start_time = time.time()


# Switch to directory for saving model parameters
os.chdir(base_dir+'/DL-NMR-Optimization/SavedParamsAndTrainingMetrics')

# Create model
model_aq = NMR_Model_Aq()

# Move the model to the GPU device
model_aq.to(device)

# Define the path to save and load the model parameters
save_path = ModelName + '_Params.pt'

# Call the function
train_losses, test_losses, is_model_trained = train_or_load_model(model_aq, train_iter, test_iter, num_epochs, save_path)


# Finish timing cell run time
end_time = time.time()
execution_time = end_time - start_time
if is_model_trained:
    np.save(ModelName + "_ExecutionTime.npy", execution_time)
    print("Execution time:", execution_time, "seconds")

No pretrained model found. Training from scratch.
Epoch [1/500], Train Loss: 23706.6727, Test Loss: 5245.3365
Epoch [2/500], Train Loss: 20304.4589, Test Loss: 4795.4088
Epoch [3/500], Train Loss: 16051.6459, Test Loss: 2917.8587
Epoch [4/500], Train Loss: 8842.2735, Test Loss: 1706.3449
Epoch [5/500], Train Loss: 5827.9281, Test Loss: 1289.0443
Epoch [6/500], Train Loss: 4698.9656, Test Loss: 1095.1946
Epoch [7/500], Train Loss: 4142.6110, Test Loss: 987.9677
Epoch [8/500], Train Loss: 3753.9534, Test Loss: 894.0563
Epoch [9/500], Train Loss: 3413.5597, Test Loss: 827.5266
Epoch [10/500], Train Loss: 3145.0858, Test Loss: 748.0352
Epoch [11/500], Train Loss: 2915.0126, Test Loss: 696.4823
Epoch [12/500], Train Loss: 2693.4374, Test Loss: 660.1776
Epoch [13/500], Train Loss: 2461.0175, Test Loss: 586.9299
Epoch [14/500], Train Loss: 2231.7419, Test Loss: 525.6307
Epoch [15/500], Train Loss: 2014.1473, Test Loss: 476.8903
Epoch [16/500], Train Loss: 1827.7409, Test Loss: 433.4988
Epoch 

Epoch [141/500], Train Loss: 586.2826, Test Loss: 146.1983
Epoch [142/500], Train Loss: 587.2560, Test Loss: 133.2175
Epoch [143/500], Train Loss: 595.1005, Test Loss: 147.2417
Epoch [144/500], Train Loss: 589.6171, Test Loss: 141.2454
Epoch [145/500], Train Loss: 587.3833, Test Loss: 149.3603
Epoch [146/500], Train Loss: 603.5394, Test Loss: 141.8135
Epoch [147/500], Train Loss: 585.2306, Test Loss: 133.5475
Epoch [148/500], Train Loss: 583.6739, Test Loss: 147.5352
Epoch [149/500], Train Loss: 586.0602, Test Loss: 146.8660
Epoch [150/500], Train Loss: 575.5677, Test Loss: 149.6967
Epoch [151/500], Train Loss: 580.1406, Test Loss: 154.8091
Epoch [152/500], Train Loss: 587.9773, Test Loss: 133.4452
Epoch [153/500], Train Loss: 570.4645, Test Loss: 148.1632
Epoch [154/500], Train Loss: 581.6007, Test Loss: 174.4433
Epoch [155/500], Train Loss: 584.6301, Test Loss: 154.2726
Epoch [156/500], Train Loss: 575.0636, Test Loss: 145.3660
Epoch [157/500], Train Loss: 569.1394, Test Loss: 134.19

Epoch [280/500], Train Loss: 467.7317, Test Loss: 126.8463
Epoch [281/500], Train Loss: 468.8760, Test Loss: 125.6320
Epoch [282/500], Train Loss: 468.5372, Test Loss: 120.2673
Epoch [283/500], Train Loss: 474.2503, Test Loss: 129.2040
Epoch [284/500], Train Loss: 458.2045, Test Loss: 123.3744
Epoch [285/500], Train Loss: 464.9212, Test Loss: 120.4864
Epoch [286/500], Train Loss: 471.3791, Test Loss: 119.9450
Epoch [287/500], Train Loss: 465.4608, Test Loss: 108.4365
Epoch [288/500], Train Loss: 463.0696, Test Loss: 116.3790
Epoch [289/500], Train Loss: 466.8753, Test Loss: 115.3829
Epoch [290/500], Train Loss: 453.9751, Test Loss: 128.4532
Epoch [291/500], Train Loss: 468.5094, Test Loss: 125.1392
Epoch [292/500], Train Loss: 467.5932, Test Loss: 121.2274
Epoch [293/500], Train Loss: 460.5262, Test Loss: 124.3561
Epoch [294/500], Train Loss: 454.5890, Test Loss: 105.8520
Epoch [295/500], Train Loss: 464.5347, Test Loss: 109.3495
Epoch [296/500], Train Loss: 458.0404, Test Loss: 110.02

Epoch [420/500], Train Loss: 412.5847, Test Loss: 101.6585
Epoch [421/500], Train Loss: 409.9798, Test Loss: 109.1084
Epoch [422/500], Train Loss: 404.7208, Test Loss: 97.5549
Epoch [423/500], Train Loss: 407.7621, Test Loss: 114.3270
Epoch [424/500], Train Loss: 405.7545, Test Loss: 118.9951
Epoch [425/500], Train Loss: 404.2813, Test Loss: 114.3992
Epoch [426/500], Train Loss: 409.2123, Test Loss: 100.5717
Epoch [427/500], Train Loss: 406.7945, Test Loss: 110.0999
Epoch [428/500], Train Loss: 405.3170, Test Loss: 110.1743
Epoch [429/500], Train Loss: 406.8397, Test Loss: 104.4276
Epoch [430/500], Train Loss: 407.7174, Test Loss: 120.8867
Epoch [431/500], Train Loss: 400.8630, Test Loss: 106.8976
Epoch [432/500], Train Loss: 395.2669, Test Loss: 94.9196
Epoch [433/500], Train Loss: 413.2180, Test Loss: 107.3143
Epoch [434/500], Train Loss: 405.1421, Test Loss: 110.5419
Epoch [435/500], Train Loss: 402.2704, Test Loss: 108.6607
Epoch [436/500], Train Loss: 401.0427, Test Loss: 104.1110

In [11]:
np.array(test_losses).min()

91.41678746044636

In [12]:
## Load training and testing datasets, validation datasets, and representative example spectra 

# Switch to directory containing datasets
os.chdir(base_dir+'/DL-NMR-Optimization/GeneratedDataAndVariables')

# Load validation dataset
spectraVal = np.load(f'Dataset44_{base_name}_ForManuscript_Val_Spec.npy')
concVal = np.load(f'Dataset44_{base_name}_ForManuscript_Val_Conc.npy')



# Load representative validation spectra and concentrations
# Load spectra of varied concentrations (all metabolites at X-mM from 0.005mm to 20mM)
ConcSpec = np.load(f'Concentration_44met_{base_name}_ForManuscript_Spec.npy')
ConcConc = np.load(f'Concentration_44met_{base_name}_ForManuscript_Conc.npy')  
#  Load uniform concentration distribution validation spectra
UniformSpec = np.load(f'UniformDist_44met_{base_name}_ForManuscript_Spec.npy')
UniformConc = np.load(f'UniformDist_44met_{base_name}_ForManuscript_Conc.npy')  
#  Load low concentration uniform concentration distribution validation spectra
LowUniformSpec = np.load(f'LowUniformDist_44met_{base_name}_ForManuscript_Spec.npy')
LowUniformConc = np.load(f'LowUniformDist_44met_{base_name}_ForManuscript_Conc.npy')
#  Load tissue mimicking concentration distribution validation spectra
MimicTissueRangeSpec = np.load(f'MimicTissueRange_44met_{base_name}_ForManuscript_Spec.npy')
MimicTissueRangeConc = np.load(f'MimicTissueRange_44met_{base_name}_ForManuscript_Conc.npy')
#  Load liver tissue mimicking concentration distribution (high relative glucose) validation spectra
MimicTissueRangeGlucSpec = np.load(f'MimicTissueRangeGluc_44met_{base_name}_ForManuscript_Spec.npy')
MimicTissueRangeGlucConc = np.load(f'MimicTissueRangeGluc_44met_{base_name}_ForManuscript_Conc.npy')
#  Load high dynamic range #2 validation spectra
HighDynamicRange2Spec = np.load(f'HighDynRange2_44met_{base_name}_ForManuscript_Spec.npy')
HighDynamicRange2Conc = np.load(f'HighDynRange2_44met_{base_name}_ForManuscript_Conc.npy') 
#  Load varied SNR validation spectra
SNR_Spec = np.load(f'SNR_44met_{base_name}_ForManuscript_Spec.npy')
SNR_Conc = np.load(f'SNR_44met_{base_name}_ForManuscript_Conc.npy')
#  Load random singlet validation spectra
Singlet_Spec = np.load(f'Singlet_44met_{base_name}_ForManuscript_Spec.npy')
Singlet_Conc = np.load(f'Singlet_44met_{base_name}_ForManuscript_Conc.npy')
#  Load random qref checker validation spectra
QrefSensSpec = np.load(f'QrefSensitivity_44met_{base_name}_ForManuscript_Spec.npy')
QrefSensConc = np.load(f'QrefSensitivity_44met_{base_name}_ForManuscript_Conc.npy')
#  Load other validation spectra
OtherValSpectra = np.load(f'OtherVal_44met_{base_name}_ForManuscript_Spec.npy')
OtherValConc = np.load(f'OtherVal_44met_{base_name}_ForManuscript_Conc.npy')




# Move the input data to the GPU device
spectraVal = torch.tensor(spectraVal).float().to(device)   
concVal = torch.tensor(concVal).float().to(device)
ConcSpec = torch.tensor(ConcSpec).float().to(device)   
ConcConc = torch.tensor(ConcConc).float().to(device)
UniformSpec = torch.tensor(UniformSpec).float().to(device)   
UniformConc = torch.tensor(UniformConc).float().to(device)
LowUniformSpec = torch.tensor(LowUniformSpec).float().to(device)   
LowUniformConc = torch.tensor(LowUniformConc).float().to(device)
MimicTissueRangeSpec = torch.tensor(MimicTissueRangeSpec).float().to(device)   
MimicTissueRangeConc = torch.tensor(MimicTissueRangeConc).float().to(device)
MimicTissueRangeGlucSpec = torch.tensor(MimicTissueRangeGlucSpec).float().to(device)   
MimicTissueRangeGlucConc = torch.tensor(MimicTissueRangeGlucConc).float().to(device)
HighDynamicRange2Spec = torch.tensor(HighDynamicRange2Spec).float().to(device)   
HighDynamicRange2Conc = torch.tensor(HighDynamicRange2Conc).float().to(device)
SNR_Spec = torch.tensor(SNR_Spec).float().to(device)   
SNR_Conc = torch.tensor(SNR_Conc).float().to(device)
Singlet_Spec = torch.tensor(Singlet_Spec).float().to(device)   
Singlet_Conc = torch.tensor(Singlet_Conc).float().to(device)
QrefSensSpec = torch.tensor(QrefSensSpec).float().to(device)   
QrefSensConc = torch.tensor(QrefSensConc).float().to(device)
OtherValSpectra = torch.tensor(OtherValSpectra).float().to(device)   
OtherValConc = torch.tensor(OtherValConc).float().to(device)

In [13]:
## Make sure best parameters are being utilized

# Switch to directory for saving model parameters
os.chdir(base_dir+'/DL-NMR-Optimization/SavedParamsAndTrainingMetrics')

# Define the path where you saved your model parameters
save_path = ModelName + '_Params.pt'

# Load the entire dictionary from the saved file
checkpoint = torch.load(save_path)

# Instantiate the model
model_aq = NMR_Model_Aq()

# Load the model's state dictionary from the loaded dictionary
model_aq.load_state_dict(checkpoint['model_state_dict'])

# Move the model to the GPU 
model_aq.to(device)

NMR_Model_Aq(
  (conv1): Conv1d(1, 42, kernel_size=(6,), stride=(1,), padding=(1,))
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2): Conv1d(42, 42, kernel_size=(6,), stride=(1,), padding=(1,))
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv3): Conv1d(42, 42, kernel_size=(6,), stride=(1,), padding=(1,))
  (pool3): MaxPool1d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv4): Conv1d(42, 42, kernel_size=(6,), stride=(1,), padding=(1,))
  (pool4): MaxPool1d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=120708, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=44, bias=True)
)

In [14]:
## Compute absolute percent error statistics on validation set

APEs = []
MAPEs = []

for i in np.arange(5000):
    GroundTruth = concVal[i].cpu().numpy()
    model_aq.eval()
    Prediction = model_aq(spectraVal[i].unsqueeze(0).unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "ValExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "ValExamples_MAPEs.npy", np.array(MAPEs))



print('Overall MAPE: ',np.array(MAPEs).mean())


Overall MAPE:  46.30652182846086


In [15]:
## Compute absolute percent error statistics on concentration varied validation spectra

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = ConcConc[i]
    model_aq.eval()
    Prediction = model_aq(ConcSpec[i].unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "ConcExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "ConcExamples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2), " - Concentrations:",ConcConc[i][0].item())

Overall MAPE:  inf
--------------------
inf  - Concentrations: 0.0
123.99  - Concentrations: 0.004999999888241291
24.44  - Concentrations: 0.02500000037252903
6.52  - Concentrations: 0.10000000149011612
2.42  - Concentrations: 0.25
1.46  - Concentrations: 0.5
1.04  - Concentrations: 1.0
0.74  - Concentrations: 2.5
0.54  - Concentrations: 10.0
0.53  - Concentrations: 20.0


In [16]:
## Compute absolute percent error statistics on uniform distribution validation spectra

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = UniformConc[i]
    model_aq.eval()
    Prediction = model_aq(UniformSpec[i].unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "UniformExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "UniformExamples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2), " - Min Value:", np.round(UniformConc[i].min().item(),4), " - Mean Value:", np.round(UniformConc[i].mean().item(),1))

Overall MAPE:  2.9033258
--------------------
0.83  - Min Value: 0.6783  - Mean Value: 9.2
19.03  - Min Value: 0.0096  - Mean Value: 10.3
1.19  - Min Value: 0.147  - Mean Value: 10.5
1.2  - Min Value: 0.5572  - Mean Value: 8.5
1.06  - Min Value: 1.3567  - Mean Value: 10.6
1.24  - Min Value: 0.6332  - Mean Value: 10.9
0.95  - Min Value: 0.7017  - Mean Value: 11.0
1.75  - Min Value: 0.3674  - Mean Value: 8.9
0.98  - Min Value: 0.8387  - Mean Value: 9.8
0.8  - Min Value: 1.0913  - Mean Value: 11.1


In [17]:
## Compute absolute percent error statistics on low concentration uniform distribution validation spectra

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = LowUniformConc[i]
    model_aq.eval()
    Prediction = model_aq(LowUniformSpec[i].unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "LowUniformExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "LowUniformExamples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2), " - Min Value:", np.round(LowUniformConc[i].min().item(),4), " - Mean Value:", np.round(LowUniformConc[i].mean().item(),1))

Overall MAPE:  8.414203
--------------------
5.77  - Min Value: 0.0111  - Mean Value: 0.1
9.98  - Min Value: 0.0103  - Mean Value: 0.1
6.98  - Min Value: 0.0153  - Mean Value: 0.1
8.47  - Min Value: 0.0117  - Mean Value: 0.1
8.68  - Min Value: 0.0089  - Mean Value: 0.1
8.67  - Min Value: 0.0075  - Mean Value: 0.1
8.5  - Min Value: 0.0117  - Mean Value: 0.1
9.12  - Min Value: 0.0052  - Mean Value: 0.1
11.31  - Min Value: 0.008  - Mean Value: 0.1
6.65  - Min Value: 0.0134  - Mean Value: 0.1


In [18]:
## Compute absolute percent error statistics on tissue mimicking distribution validation spectra

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = MimicTissueRangeConc[i]
    model_aq.eval()
    Prediction = model_aq(MimicTissueRangeSpec[i].unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "MimicTissueRangeExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "MimicTissueRangeExamples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2), " - Min Value:", np.round(MimicTissueRangeConc[i].min().item(),4), " - Mean Value:", np.round(MimicTissueRangeConc[i].mean().item(),1))

Overall MAPE:  9.012616
--------------------
16.29  - Min Value: 0.008  - Mean Value: 0.8
11.34  - Min Value: 0.009  - Mean Value: 0.9
11.78  - Min Value: 0.0138  - Mean Value: 1.5
9.06  - Min Value: 0.0107  - Mean Value: 0.7
6.29  - Min Value: 0.0191  - Mean Value: 0.7
11.51  - Min Value: 0.0186  - Mean Value: 0.8
5.49  - Min Value: 0.0175  - Mean Value: 0.8
5.68  - Min Value: 0.0238  - Mean Value: 1.3
5.77  - Min Value: 0.0168  - Mean Value: 0.7
6.92  - Min Value: 0.0171  - Mean Value: 0.9


In [19]:
## Compute absolute percent error statistics on tissue mimicking distribution validation spectra (high relative glucose concentration)

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = MimicTissueRangeGlucConc[i]
    model_aq.eval()
    Prediction = model_aq(MimicTissueRangeGlucSpec[i].unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "MimicTissueRangeGlucExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "MimicTissueRangeGlucExamples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2), " - Min Value:", np.round(MimicTissueRangeGlucConc[i].min().item(),4), " - Mean Value:", np.round(MimicTissueRangeGlucConc[i].mean().item(),1))

Overall MAPE:  15.310718
--------------------
8.13  - Min Value: 0.013  - Mean Value: 0.6
20.07  - Min Value: 0.0115  - Mean Value: 0.4
13.59  - Min Value: 0.0115  - Mean Value: 0.4
13.44  - Min Value: 0.0115  - Mean Value: 0.6
16.45  - Min Value: 0.0115  - Mean Value: 1.0
18.82  - Min Value: 0.0115  - Mean Value: 1.1
22.01  - Min Value: 0.0115  - Mean Value: 0.8
12.27  - Min Value: 0.0115  - Mean Value: 0.5
10.32  - Min Value: 0.0115  - Mean Value: 0.5
18.0  - Min Value: 0.0115  - Mean Value: 1.1


In [20]:
## Compute absolute percent error statistics on a further high dynamic range dataset

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = HighDynamicRange2Conc[i]
    model_aq.eval()
    Prediction = model_aq(HighDynamicRange2Spec[i].unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "HighDynamicRange2Examples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "HighDynamicRange2Examples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2), " - Min Value:", np.round(HighDynamicRange2Conc[i].min().item(),4), " - Mean Value:", np.round(HighDynamicRange2Conc[i].mean().item(),1))

Overall MAPE:  65.88866
--------------------
65.05  - Min Value: 0.0062  - Mean Value: 2.1
67.91  - Min Value: 0.006  - Mean Value: 3.7
41.63  - Min Value: 0.0066  - Mean Value: 4.3
62.76  - Min Value: 0.0094  - Mean Value: 4.3
62.19  - Min Value: 0.0068  - Mean Value: 4.9
75.36  - Min Value: 0.005  - Mean Value: 3.8
59.07  - Min Value: 0.0101  - Mean Value: 3.2
51.39  - Min Value: 0.0062  - Mean Value: 3.2
75.66  - Min Value: 0.0053  - Mean Value: 5.3
97.85  - Min Value: 0.0054  - Mean Value: 2.5


In [21]:
## Compute absolute percent error statistics on a examples of varying SNR

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = 0.43
    model_aq.eval()
    Prediction = model_aq(SNR_Spec[i].unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth - Prediction_cpu[0][metabolite]) / GroundTruth
        APE.append(abs(per_err))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "SNR_Examples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "SNR_Examples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2))

Overall MAPE:  1.7120953543745692
--------------------
1.58
1.61
1.68
1.6
1.63
1.59
1.78
1.82
1.74
2.11


In [22]:
## Compute absolute percent error statistics on a dataset with singlets added at random

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = 0.43
    model_aq.eval()
    Prediction = model_aq(Singlet_Spec[i].unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth - Prediction_cpu[0][metabolite]) / GroundTruth
        APE.append(abs(per_err))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "SingletExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "SingletExamples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2))

Overall MAPE:  10.175205001304317
--------------------
1.65
1.61
1.66
2.85
10.53
10.6
10.51
17.75
19.89
24.7


In [23]:
## Compute absolute percent error statistics on a examples of varying SNR

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = 0.43
    model_aq.eval()
    Prediction = model_aq(QrefSensSpec[i].unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth - Prediction_cpu[0][metabolite]) / GroundTruth
        APE.append(abs(per_err))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "QrefSensitivity_Examples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "QrefSensitivity_Examples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2))

Overall MAPE:  47.699623025193745
--------------------
3.6
13.08
22.87
32.73
42.67
52.53
62.43
72.41
82.39
92.28


In [24]:
Pred = model_aq(OtherValSpectra[0].unsqueeze(2))
Pred[0][Pred[0] < 0] = 0
print("Sinusoidal Baseline 1")
print(Pred[0])
print("___________")
print("___________")

Pred = model_aq(OtherValSpectra[1].unsqueeze(2))
Pred[0][Pred[0] < 0] = 0
print("Sinusoidal Baseline 2")
print(Pred[0])
print("___________")
print("___________")

Pred = model_aq(OtherValSpectra[2].unsqueeze(2))
Pred[0][Pred[0] < 0] = 0
print("HD-Range 1 - 1s and 20s")
print(Pred[0])

Pred = model_aq(OtherValSpectra[3].unsqueeze(2))
Pred[0][Pred[0] < 0] = 0
print("HD-Range 2 - 0s and 20s")
print(Pred[0])

Sinusoidal Baseline 1
tensor([0.4844, 0.5548, 0.3623, 0.5457, 0.5170, 0.4948, 0.4922, 0.6211, 0.6265,
        0.3860, 0.4015, 0.6617, 0.4961, 0.9399, 0.5970, 0.6553, 0.6393, 0.4767,
        0.5241, 0.3757, 0.5376, 0.8830, 0.2744, 0.5080, 0.6039, 0.4369, 0.6109,
        0.5302, 0.5776, 0.4771, 0.4061, 0.8188, 0.5536, 0.3928, 0.3995, 0.5282,
        0.3969, 0.4007, 0.4839, 0.3778, 0.3974, 0.4550, 0.5753, 0.4245],
       device='cuda:0', grad_fn=<SelectBackward0>)
___________
___________
Sinusoidal Baseline 2
tensor([0.3809, 0.5881, 0.0000, 0.5474, 0.4380, 0.4486, 0.4283, 0.2107, 0.4857,
        0.6011, 0.4262, 0.5570, 0.8225, 0.2587, 0.6783, 0.1008, 0.8347, 0.5361,
        0.5018, 0.4002, 0.3517, 0.3767, 0.6085, 0.5943, 0.4307, 0.4681, 0.2294,
        0.0741, 0.5909, 0.3842, 0.3713, 0.2265, 0.1692, 0.4498, 0.6130, 0.3550,
        0.6212, 0.7562, 0.6920, 0.3371, 0.4318, 0.8393, 0.2896, 0.1432],
       device='cuda:0', grad_fn=<SelectBackward0>)
___________
___________
HD-Range 1 - 1s and 